In [ ]:
%env ANYWIDGET_HMR=1

# Show3D: Complete Feature Demo

Demonstrates all Show3D features using realistic electron microscopy synthetic data.

In [ ]:
import numpy as np
from quantem.widget import Show3D


def make_focal_series(n_frames=30, size=256):
    """Through-focus series: nanoparticles with Fresnel fringes at edges."""
    y, x = np.mgrid[:size, :size]
    particles = [
        (size * 0.35, size * 0.4, 18, 1.0),
        (size * 0.65, size * 0.55, 25, 0.7),
        (size * 0.45, size * 0.7, 12, 1.2),
        (size * 0.7, size * 0.3, 15, 0.9),
    ]
    defocus = np.linspace(-60, 60, n_frames)
    frames = np.zeros((n_frames, size, size), dtype=np.float32)
    for f_idx, df in enumerate(defocus):
        frame = np.full((size, size), 0.5)
        for cx, cy, r, z in particles:
            dist = np.sqrt((x - cx) ** 2 + (y - cy) ** 2)
            edge = 1.0 / (1 + np.exp((dist - r) * 2))
            if abs(df) > 3:
                fresnel = np.cos(0.005 * df * (dist - r) ** 2) * np.exp(
                    -((dist - r) ** 2) / (2 * (3 + abs(df) * 0.15) ** 2)
                )
                frame += z * (edge * 0.3 + fresnel * 0.2 * np.sign(df))
            else:
                frame += z * edge * 0.4
        frame += np.random.normal(0, 0.03, (size, size))
        frames[f_idx] = frame
    return frames


def make_insitu_growth(n_frames=40, size=128):
    """Nanoparticle nucleation and growth over time."""
    y, x = np.mgrid[:size, :size]
    sites = [(30, 40, 5), (80, 60, 8), (50, 90, 12), (100, 100, 3), (60, 30, 18)]
    frames = np.zeros((n_frames, size, size), dtype=np.float32)
    for f in range(n_frames):
        frame = np.random.normal(0.1, 0.02, (size, size))
        for cx, cy, t_start in sites:
            if f >= t_start:
                growth = min(1.0, (f - t_start) / 15)
                radius = 3 + growth * 12
                dist = np.sqrt((x - cx) ** 2 + (y - cy) ** 2)
                frame += (0.5 + 0.3 * growth) * np.exp(-dist ** 2 / (2 * radius ** 2))
        frames[f] = frame
    return frames


def make_lattice_rotation(n_frames=20, size=128):
    """Crystal with rotating lattice fringes (grain rotation in-situ)."""
    y, x = np.mgrid[:size, :size]
    frames = np.zeros((n_frames, size, size), dtype=np.float32)
    for f in range(n_frames):
        angle = f * np.pi / (2 * n_frames)
        freq = 0.08
        frames[f] = np.cos(
            2 * np.pi * freq * (x * np.cos(angle) + y * np.sin(angle))
        )
        frames[f] += 0.5 * np.cos(
            2 * np.pi * freq * 1.5 * (
                x * np.cos(angle + np.pi / 3) + y * np.sin(angle + np.pi / 3)
            )
        )
        frames[f] += np.random.normal(0, 0.2, (size, size))
    return frames.astype(np.float32)


def make_haadf_stack(n_frames=25, size=128):
    """HAADF-STEM image stack with Z-contrast columns and scan noise."""
    y, x = np.mgrid[:size, :size]
    # Simulate atomic columns on a grid
    columns = []
    spacing = 16
    for row in range(4, size - 4, spacing):
        for col in range(4, size - 4, spacing):
            z_contrast = np.random.uniform(0.4, 1.0)
            columns.append((col, row, z_contrast))
    frames = np.zeros((n_frames, size, size), dtype=np.float32)
    for f in range(n_frames):
        frame = np.random.normal(0.05, 0.01, (size, size))
        # Scan distortion varies per frame
        dx = np.random.normal(0, 0.3)
        dy = np.random.normal(0, 0.3)
        for cx, cy, z in columns:
            dist2 = (x - cx - dx) ** 2 + (y - cy - dy) ** 2
            frame += z * np.exp(-dist2 / (2 * 2.5 ** 2))
        # Add a bright contamination region in upper-right
        bright_dist = np.sqrt((x - size * 0.8) ** 2 + (y - size * 0.2) ** 2)
        frame += 0.6 * np.exp(-bright_dist ** 2 / (2 * 15 ** 2))
        frames[f] = frame
    return frames


def make_hdr_stack(n_frames=20, size=128):
    """High dynamic range stack (bright diffraction spots on dark background)."""
    y, x = np.mgrid[:size, :size]
    frames = np.zeros((n_frames, size, size), dtype=np.float32)
    cx, cy = size // 2, size // 2
    for f in range(n_frames):
        frame = np.random.exponential(0.5, (size, size)).astype(np.float32)
        # Central beam
        dist_center = (x - cx) ** 2 + (y - cy) ** 2
        frame += 1000 * np.exp(-dist_center / (2 * 3 ** 2))
        # Diffraction spots that vary with frame
        angle_offset = f * np.pi / (4 * n_frames)
        for k in range(6):
            theta = k * np.pi / 3 + angle_offset
            sx = cx + 30 * np.cos(theta)
            sy = cy + 30 * np.sin(theta)
            d2 = (x - sx) ** 2 + (y - sy) ** 2
            frame += 200 * np.exp(-d2 / (2 * 2 ** 2))
        frames[f] = frame
    return frames


print("Data generators ready.")

## 1. Basic Stack (NumPy)

Through-focus series of nanoparticles. Fresnel fringes appear at particle edges when defocused.

In [ ]:
focal_stack = make_focal_series(n_frames=30, size=256)
defocus_values = np.linspace(-60, 60, 30)
labels = [f"C10={df:.0f} nm" for df in defocus_values]

Show3D(
    focal_stack,
    labels=labels,
    title="Through-Focus Series: Nanoparticles",
    cmap="gray",
)

## 2. PyTorch Tensor

Same focal series converted to a PyTorch tensor. Show3D accepts both NumPy and PyTorch.

In [ ]:
import torch

focal_torch = torch.from_numpy(focal_stack)
print(f"Tensor shape: {focal_torch.shape}, dtype: {focal_torch.dtype}")

Show3D(
    focal_torch,
    labels=labels,
    title="Through-Focus (PyTorch Tensor)",
    cmap="gray",
)

## 3. Playback Controls

Use `.play()`, `.pause()`, and `.stop()` to control playback programmatically.

In [ ]:
w_playback = Show3D(
    focal_stack,
    labels=labels,
    title="Playback Demo -- use play/pause/stop buttons",
    cmap="gray",
    fps=10,
)
w_playback

In [ ]:
# Programmatic control:
# w_playback.play()   # Start playing
# w_playback.pause()  # Pause
# w_playback.stop()   # Stop and reset to frame 0

## 4. Custom FPS, Loop Range, Reverse

In-situ nanoparticle nucleation and growth. Loop over just the nucleation burst (frames 3-20), play in reverse.

In [ ]:
growth_stack = make_insitu_growth(n_frames=40, size=128)

w_growth = Show3D(
    growth_stack,
    title="In-Situ Nanoparticle Growth",
    cmap="inferno",
    fps=12,
)
w_growth.reverse = True
w_growth.loop_start = 3
w_growth.loop_end = 20
w_growth

## 5. Labels and Timestamps

Physical time labels for each frame of the in-situ growth experiment.

In [ ]:
n_growth = 40
time_seconds = np.linspace(0, 120, n_growth)  # 2 minutes total
time_labels = [f"t={t:.1f} s" for t in time_seconds]

Show3D(
    growth_stack,
    labels=time_labels,
    title="In-Situ Growth with Timestamps",
    cmap="inferno",
    timestamps=time_seconds.tolist(),
    timestamp_unit="s",
    fps=8,
)

## 6. ROI -- Circle

Circular ROI on the HAADF stack, placed over the bright contamination region.

In [ ]:
haadf_stack = make_haadf_stack(n_frames=25, size=128)

w_roi_circle = Show3D(
    haadf_stack,
    title="HAADF Stack -- Circle ROI on bright region",
    cmap="viridis",
)
# Place circle ROI over the bright contamination spot (upper-right)
w_roi_circle.set_roi(x=102, y=26, radius=18)
w_roi_circle.roi_shape = "circle"
w_roi_circle

## 7. ROI -- Square and Rectangle

Square and rectangle ROI shapes on the same HAADF data.

In [ ]:
w_roi_sq = Show3D(
    haadf_stack,
    title="HAADF Stack -- Square ROI",
    cmap="viridis",
)
w_roi_sq.set_roi(x=64, y=64, radius=20)
w_roi_sq.roi_shape = "square"
w_roi_sq

In [ ]:
w_roi_rect = Show3D(
    haadf_stack,
    title="HAADF Stack -- Rectangle ROI",
    cmap="viridis",
)
w_roi_rect.roi_active = True
w_roi_rect.roi_shape = "rectangle"
w_roi_rect.roi_x = 80
w_roi_rect.roi_y = 50
w_roi_rect.roi_width = 40
w_roi_rect.roi_height = 20
w_roi_rect

## 8. FFT Panel

Crystal lattice with rotating fringes (simulating grain rotation during in-situ heating). The FFT panel reveals the changing spot pattern as the lattice rotates.

In [ ]:
lattice_stack = make_lattice_rotation(n_frames=20, size=128)
angle_labels = [f"{a:.1f} deg" for a in np.linspace(0, 90, 20)]

Show3D(
    lattice_stack,
    labels=angle_labels,
    title="Lattice Rotation -- FFT shows spot migration",
    cmap="gray",
    show_fft=True,
    fps=4,
)

## 9. Comparison Mode

Side-by-side comparison of the in-focus frame vs a strongly defocused frame.

In [ ]:
# Frame 15 is near focus (C10~0 nm), frame 0 is strongly underfocused (C10=-60 nm)
w_compare = Show3D(
    focal_stack,
    labels=labels,
    title="Comparison: browse left, reference right",
    cmap="gray",
)
w_compare.compare_with(15)  # Reference = in-focus frame
w_compare

## 10. Colormaps

Different colormaps applied to the HAADF-like Z-contrast stack.

In [ ]:
Show3D(
    haadf_stack,
    title="HAADF Stack -- inferno colormap",
    cmap="inferno",
)

## 11. Log Scale + Auto Contrast

High dynamic range diffraction pattern stack. The central beam is ~1000x brighter than the background. Log scale and auto-contrast reveal the weak diffraction spots.

In [ ]:
hdr_stack = make_hdr_stack(n_frames=20, size=128)
print(f"HDR range: [{hdr_stack.min():.1f}, {hdr_stack.max():.1f}]")

Show3D(
    hdr_stack,
    title="Diffraction Stack -- log scale + auto contrast",
    cmap="hot",
    log_scale=True,
    auto_contrast=True,
    percentile_low=2.0,
    percentile_high=99.5,
)

## 12. Scale Bar

Focal series with a calibrated pixel size of 0.25 nm/px (typical HRTEM).

In [ ]:
Show3D(
    focal_stack,
    labels=labels,
    title="HRTEM Focal Series -- 0.25 nm/px",
    cmap="gray",
    pixel_size=0.25,
    scale_bar_visible=True,
    scale_bar_length_px=80,
    scale_bar_thickness_px=5,
    scale_bar_font_size_px=14,
)

## 13. Boomerang (Ping-Pong)

Plays forward then backward. Useful for oscillating phenomena like beam-induced motion.

In [ ]:
w_boom = Show3D(
    growth_stack,
    title="In-Situ Growth -- Boomerang mode",
    cmap="inferno",
    fps=15,
)
w_boom.boomerang = True
w_boom

## 14. Bookmarks

Mark interesting defocus values: the in-focus frame and the two extremes where Fresnel fringes are strongest.

In [ ]:
w_bm = Show3D(
    focal_stack,
    labels=labels,
    title="Focal Series -- Bookmarked frames",
    cmap="gray",
)
# Bookmark: underfocus extreme, in-focus, overfocus extreme
w_bm.bookmarked_frames = [0, 15, 29]
w_bm

## 15. Manual vmin/vmax

Clip the display range to highlight subtle contrast differences in the growth series.

In [ ]:
Show3D(
    growth_stack,
    title="In-Situ Growth -- Manual vmin/vmax",
    cmap="inferno",
    vmin=0.0,
    vmax=0.5,
)

## 16. Hide Stats

Clean view with the statistics panel hidden.

In [ ]:
Show3D(
    lattice_stack,
    labels=angle_labels,
    title="Lattice Rotation -- stats hidden",
    cmap="gray",
    show_stats=False,
)